# Magnetics Sources : 
https://dataservices.gfz-potsdam.de/intermagnet/showshort.php?id=cfc7a77f-12cc-11ef-967a-4ffbfe06208e

https://resources.vic.gov.au/geology-exploration/maps-reports-data/geophysics

https://resources.vic.gov.au/geology-exploration/maps-reports-data/geophysics


### To Do : use Qgis to make polygons -> cut out sections 


```
Find lithologies of an area 
Make some cut outs 
Train
Test etc.


```